In [1]:
#import SparkSession
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1589653496572_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#create spark session object
spark=SparkSession.builder.appName('proyecto3').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589653496572_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589653496572_0002,pyspark,idle,Link,Link,✔


In [5]:
# carga de datos csv en spark desde un bucket S3.
colombia=spark.read.csv('s3://proyectotres/colombia/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

m_muertos_dias =spark.read.csv('s3://proyectotres/mundial/time_series_covid19_deaths_global_iso3_regions.csv',inferSchema=True,header=True)

m_confirm_dias =spark.read.csv('s3://proyectotres/mundial/time_series_covid19_confirmed_global_iso3_regions.csv',inferSchema=True,header=True)

m_recup_dias =spark.read.csv('s3://proyectotres/mundial/time_series_covid19_recovered_global_iso3_regions.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
#Promedio de edad de muertos por departamento de mayor a menor

edadPromedio = colombia.filter(colombia['Estado']=='Fallecido').groupby('Departamento o Distrito ').agg({'Edad': 'mean'}).sort('avg(Edad)', ascending=False)
edadPromedio.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+------------------+
|Departamento o Distrito |         avg(Edad)|
+------------------------+------------------+
|               Antioquia| 81.16666666666667|
|                 Caquetá|              79.0|
|                 Quindio|              78.0|
|               Magdalena|            77.125|
|      Norte de Santander|              75.7|
|                 Córdoba|              75.0|
|                  Caldas|              74.5|
|                   Huila|             73.25|
|    Santa Marta D.T. ...| 70.82352941176471|
|       Buenaventura D.E.| 70.66666666666667|
|                    Meta|              70.5|
|                Amazonas|             69.25|
|                   Cesar| 68.81818181818181|
|               Risaralda|              67.5|
|                 Bolívar|              67.5|
|     Cartagena D.T. y C.| 67.37254901960785|
|             Bogotá D.C.| 67.21518987341773|
|         Valle del Cauca|           66.6125|
|       Barranquilla D.E.|        

In [96]:
#Recuperados en cada departamento por sexo
colombia.filter(colombia['Estado']=='Recuperado').groupBy('Sexo', 'Departamento o distrito ').count().sort('Departamento o distrito ', ascending=True).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------------+-----+
|Sexo|Departamento o distrito |count|
+----+------------------------+-----+
|   F|                Amazonas|   10|
|   M|                Amazonas|   11|
|   M|               Antioquia|  149|
|   F|               Antioquia|  156|
|   M|    Archipiélago de S...|    4|
|   M|               Atlántico|   14|
|   F|               Atlántico|   15|
|   M|       Barranquilla D.E.|   29|
|   F|       Barranquilla D.E.|   32|
|   F|             Bogotá D.C.|  629|
|   M|             Bogotá D.C.|  607|
|   F|                 Bolívar|    7|
|   M|                 Bolívar|    9|
|   M|                  Boyacá|   13|
|   F|                  Boyacá|   11|
|   M|       Buenaventura D.E.|    3|
|   F|       Buenaventura D.E.|    3|
|   M|                  Caldas|   18|
|   F|                  Caldas|   23|
|   F|     Cartagena D.T. y C.|   77|
+----+------------------------+-----+
only showing top 20 rows

In [124]:
#Estado de las mujeres infectadas en Colombia
colombia.filter(colombia['Sexo']=='F').groupBy('Estado').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|      Estado|count|
+------------+-----+
|        null|    3|
|  Recuperado| 1591|
|        Leve| 3226|
|Asintomático|  317|
|       Grave|   41|
|   Fallecido|  197|
|    Moderado|  259|
|        leve|    3|
+------------+-----+

In [127]:
#Cuantos confirmados hay en Colombia por tipo de contagio ordenados de mayor a menor
colombia.groupBy('Tipo').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
|       Tipo|count|
+-----------+-----+
| En estudio| 8231|
|Relacionado| 3859|
|  Importado|  840|
+-----------+-----+

In [128]:
#Cuantos recuperados hay en Colombia por departamento de mayor a menor
colombia.filter(colombia['Estado']=='Recuperado').groupBy('Departamento o Distrito ').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-----+
|Departamento o Distrito |count|
+------------------------+-----+
|             Bogotá D.C.| 1236|
|         Valle del Cauca|  451|
|               Antioquia|  305|
|            Cundinamarca|  157|
|     Cartagena D.T. y C.|  148|
|               Risaralda|  139|
|       Barranquilla D.E.|   61|
|                   Huila|   61|
|    Santa Marta D.T. ...|   60|
|      Norte de Santander|   59|
|                 Quindio|   56|
|                   Cesar|   45|
|                    Meta|   42|
|                  Caldas|   41|
|                  Tolima|   40|
|                  Nariño|   37|
|               Santander|   30|
|               Atlántico|   29|
|                  Boyacá|   24|
|                   Cauca|   21|
+------------------------+-----+
only showing top 20 rows

In [137]:
#Muertos en el país con mayor cantidad en cada continente ordenados de mayor a menor

ta = m_muertos_dias.select('Country/Region','5/13/20','Region Name').groupBy('Region Name').max('5/13/20').sort('max(5/13/20)', ascending=False)
tb = m_muertos_dias.select('Country/Region','5/13/20','Region Name').groupBy('Country/Region').max('5/13/20').sort('max(5/13/20)', ascending=False)

inner_join = ta.join(tb, ta['max(5/13/20)'] == tb['max(5/13/20)'])
inner_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+----------------+------------+
|Region Name|max(5/13/20)|  Country/Region|max(5/13/20)|
+-----------+------------+----------------+------------+
|   Americas|       84119|              US|       84119|
|     Europe|       33186|  United Kingdom|       33186|
|       Asia|        6783|            Iran|        6783|
|     Africa|         556|           Egypt|         556|
|    Oceania|          45|         Tunisia|          45|
|    Oceania|          45|       Australia|          45|
|       null|          13|Diamond Princess|          13|
+-----------+------------+----------------+------------+

In [125]:
#consultamos cuantos muertos hay por país hasta el día 13/05/2020 de mayor a menor
m_muertos_dias.select('Country/Region','5/13/20').groupBy('Country/Region').sum().orderBy('sum(5/13/20)', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------+
|Country/Region|sum(5/13/20)|
+--------------+------------+
|            US|       84119|
|United Kingdom|       33264|
|         Italy|       31106|
|         Spain|       27104|
|        France|       27077|
|        Brazil|       13240|
|       Belgium|        8843|
|       Germany|        7861|
|          Iran|        6783|
|   Netherlands|        5581|
|        Canada|        5425|
|         China|        4637|
|        Mexico|        4220|
|        Turkey|        3952|
|        Sweden|        3460|
|         India|        2551|
|       Ecuador|        2334|
|        Russia|        2212|
|          Peru|        2169|
|   Switzerland|        1870|
+--------------+------------+
only showing top 20 rows

In [126]:
#Cuál fue el primer continente con recuperados y cuántos
m_recup_dias.filter(m_recup_dias['1/22/20']!='0').select('Region Name', '1/22/20').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------+
|Region Name|1/22/20|
+-----------+-------+
|       Asia|     28|
+-----------+-------+